In [170]:
import sys
sys.path.append('../../src')

In [171]:
import pandas
import numpy
from energiapy.components.temporal_scale import TemporalScale
from energiapy.components.resource import Resource, VaryingResource
from energiapy.components.process import Process, VaryingProcess
from energiapy.components.material import Material
from energiapy.components.location import Location
from energiapy.components.network import Network
from energiapy.components.scenario import Scenario
from energiapy.components.transport import Transport
from energiapy.model.formulate import formulate, Constraints, Objective, ModelClass
from energiapy.utils.nsrdb_utils import fetch_nsrdb_data
from energiapy.plot import plot_results, plot_scenario
from energiapy.plot.plot_results import CostY, CostX
from energiapy.model.solve import solve
from itertools import product

from energiapy.components.result import Result
import pandas
from itertools import product
from matplotlib import rc
import matplotlib.pyplot as plt
from energiapy.model.solve import solve
from energiapy.plot.plot_results import CostY, CostX
from energiapy.components.location import Location
from energiapy.plot import plot_results, plot_scenario
from energiapy.utils.nsrdb_utils import fetch_nsrdb_data
from energiapy.model.formulate import formulate, Constraints, Objective
from energiapy.model.bounds import CapacityBounds
from energiapy.utils.data_utils import get_data, make_henry_price_df, remove_outliers, load_results
from energiapy.components.transport import Transport
from energiapy.components.scenario import Scenario
from energiapy.components.network import Network
from energiapy.components.location import Location
from energiapy.components.material import Material

from energiapy.components.process import Process, VaryingProcess
from energiapy.components.resource import Resource, VaryingResource
from energiapy.components.temporal_scale import TemporalScale
import numpy
from energiapy.conversion.photovoltaic import solar_power_output
from energiapy.conversion.windmill import wind_power_output
from energiapy.aggregation.reduce_scenario import reduce_scenario, Clustermethod
from energiapy.aggregation.ahc import agg_hierarchial_elbow, IncludeAHC, Fit
import matplotlib.pyplot as plt

from energiapy.model.weights import EmissionWeights


\begin{align}
\min_x \quad \theta^TH^Tx& + c^Tx\\
\text{s.t.} \quad Ax &\leq b + F\theta\\
A_{eq}x &= b_{eq}\\
A_\theta \theta &\leq b_\theta\\
x &\in R^n\\
\end{align}


In [172]:
scales = TemporalScale(discretization_list=[1])

In [173]:
bigM = 10**15
smallM = 0.0000000001

Declare Materials

In [174]:

LithiumCarbonateR = Material(name='LithiumCarbonateR', gwp=10602.89995, odp=0.0005311, acid=83.2005862, eutt=185.9710578, eutf=6.0273706, eutm=42.7091789, basis='kg CO2 eq./ton, kg CFC-11 eq./ton, kg SO2 eq./ton, mol N eq./ton, kg P eq./ton, kg N eq./ton', label='Rock-based Lithium',
                citation='https://www.iea.org/data-and-statistics/charts/ghg-emissions-intensity-for-lithium-by-resource-type-and-processing-route')  
LithiumCarbonateB = Material(name='LithiumCarbonateB', gwp=2147.78164, odp=0.0000323, acid=14.7111022, eutt=55.7722302, eutf=1.9709617, eutm=8.3834083, basis='kg CO2 eq./ton, kg CFC-11 eq./ton, kg SO2 eq./ton, mol N eq./ton, kg P eq./ton, kg N eq./ton', label='Brine-based Lithium',
               citation='https://www.iea.org/data-and-statistics/charts/ghg-emissions-intensity-for-lithium-by-resource-type-and-processing-route')  

Steel = Material(name ='steel', gwp= 2121.152427, odp=0.0000524, acid=7.071973, eutt=20.1478856, eutf=0.9654292, eutm=2.0671433, basis='kg CO2 eq./ton, kg CFC-11 eq./ton, kg SO2 eq./ton, mol N eq./ton, kg P eq./ton, kg N eq./ton', label = 'Steel', 
                 citation= 'Annika Eberle et al., (2023), Materials used in U.S. Wind Energy Technologies: Quanitities and Availability for Two Future Scenarios')

Cast_iron = Material(name ='cast iron', gwp= 1759.710573, odp=0.0000443, acid=6.0061499, eutt=18.2339132, eutf=0.66155, eutm=1.7330535, basis='kg CO2 eq./ton, kg CFC-11 eq./ton, kg SO2 eq./ton, mol N eq./ton, kg P eq./ton, kg N eq./ton', label = 'Cast iron', 
                 citation= 'Annika Eberle et al., (2023), Materials used in U.S. Wind Energy Technologies: Quanitities and Availability for Two Future Scenarios')

Concrete = Material(name ='concrete', gwp= 90.91, odp=0.00000040125, acid=0.246868, eutt=1.546130613, eutf=0.015425534, eutm=0.140321324, basis='kg CO2 eq./ton, kg CFC-11 eq./ton, kg SO2 eq./ton, mol N eq./ton, kg P eq./ton, kg N eq./ton', label = 'Concrete', 
                 citation= 'Annika Eberle et al., (2023), Materials used in U.S. Wind Energy Technologies: Quanitities and Availability for Two Future Scenarios')

Glass = Material(name = 'glass', gwp = 1118.5, odp=0.0000141, acid=8.8362743, eutt=22.0973602, eutf=0.1263297, eutm=1.8534905, basis='kg CO2 eq./ton, kg CFC-11 eq./ton, kg SO2 eq./ton, mol N eq./ton, kg P eq./ton, kg N eq./ton', label = 'glass',
                 citation = 'Statista.com/statistics/1118670/materials-used-manufacture-solar-pv-plant-globally. & https://solaredition.com/raw-materials-breakdown-for-building-a-1-megawatt-solar-photovoltaic-plant-2017')

Aluminium = Material(name = 'aluminium', gwp = 5632.53, odp=0.0000578, acid=31.2744756, eutt=65.5836037, eutf=1.8348121, eutm=6.2774955, basis='kg CO2 eq./ton, kg CFC-11 eq./ton, kg SO2 eq./ton, mol N eq./ton, kg P eq./ton, kg N eq./ton', label = 'aluminium',
                 citation = 'https://www.researchgate.net/figure/Overall-environmental-impact-per-kg-Pts-kg-and-GWP-kg-of-CO2-eq-kg-aluminium-of_fig2_295243557. & Statista.com/statistics/1118670/materials-used-manufacture-solar-pv-plant-globally. & https://solaredition.com/raw-materials-breakdown-for-building-a-1-megawatt-solar-photovoltaic-plant-2017')

Silicon = Material(name = 'silicon', gwp = 50137.8, odp=0.0012152, acid=188.0829961, eutt=469.8237153, eutf=18.9483911, eutm=46.3346166, basis='kg CO2 eq./ton, kg CFC-11 eq./ton, kg SO2 eq./ton, mol N eq./ton, kg P eq./ton, kg N eq./ton', label = 'silicon',
                 citation = 'Statista.com/statistics/1118670/materials-used-manufacture-solar-pv-plant-globally. & https://solaredition.com/raw-materials-breakdown-for-building-a-1-megawatt-solar-photovoltaic-plant-2017')

Copper = Material(name = 'copper', gwp = 8660.4, odp=0.0003416, acid=516.3936129, eutt=409.7976693, eutf=46.0940178, eutm=29.6096174, basis='kg CO2 eq./ton, kg CFC-11 eq./ton, kg SO2 eq./ton, mol N eq./ton, kg P eq./ton, kg N eq./ton', label = 'copper',
                 citation = 'Statista.com/statistics/1118670/materials-used-manufacture-solar-pv-plant-globally. & https://solaredition.com/raw-materials-breakdown-for-building-a-1-megawatt-solar-photovoltaic-plant-2017')


Declare Processes

In [175]:
Solar = Resource(name='Solar', cons_max=100, basis='MW', label='Solar Power')

Wind = Resource(name='Wind', cons_max=100, basis='MW', label='Wind Power')

Power = Resource(name='Power', basis='MW', label='Power generated', demand = True, varying = [VaryingResource.UNCERTAIN_DEMAND], varying_bounds  = (0,1))#(min(demand['COAST']), 1))

# H2O = Resource(name='H2O', cons_max=100, basis='tons', label='Water', varying= [VaryingResource.UNCERTAIN_AVAILABILITY], varying_bounds= (0.3, 0.95))
H2O = Resource(name='H2O', cons_max=500, basis='tons', label='Water')

O2 = Resource(name='O2', sell=True, basis='tons', label='Oxygen')

H2 = Resource(name='H2', basis='tons', label='Hydrogen', demand= True) #, varying = [VaryingResource.UNCERTAIN_DEMAND], varying_bounds  = (0.6, 0.8))

Uranium = Resource(name='Uranium', cons_max=1000,
                   price=42.70/(250/2), basis='kg', label='Uranium')

LiI = Process(name='LiI', storage=Power, vopex=1302, capex=0, fopex=0,  prod_max=50,
              storage_loss=0.11,  label='Lithium-ion battery', basis='MW', store_max=100)

PEM = Process(name='PEM', conversion={Power: -1, H2: 0.3537, H2O: -3.1839}, capex=0, fopex=0,
              vopex=1.55*10**3, prod_max=100, prod_min = 0, basis = 'tons', label = 'PEM' )


WF = Process(name='WF', conversion={Wind: -1, Power: 0.85}, vopex=990, capex=0, fopex=0,
             prod_max=100, label='Wind mill array', basis='MW', varying = [VaryingProcess.UNCERTAIN_CAPACITY])

PV = Process(name='PV', conversion={Solar: -1, Power: 0.75}, vopex=567, capex=0, fopex=0,
             prod_max=100, label='Solar PV', basis='MW', varying = [VaryingProcess.UNCERTAIN_CAPACITY])


ASMR = Process(name='ASMR', conversion={Uranium: -4.17*10**(-5), H2O: -3.364, Power: 1},
               capex=7988951, fopex=0.04*0.730, vopex=0, prod_max=50, label='Small modular reactors (SMRs)', block='power_gen')

# region = Location(name='region', processes={LiI, WF, PV}, scales=scales, label='some region wo dispatchable')

region = Location(name='region', processes={WF, PV, PEM}, scales=scales, label='some region')

scenario = Scenario(name='scenario', demand={region: {Power:100, H2: 20}}, network=region, scales=scales, label='example scenario')

matrix = formulate(scenario=scenario, model_class=ModelClass.MPLP)

The problem has the following variables:
Resource inventory level (Inv) x 0
Exact resource discharge (Sf) x 1
Uncertain resource discharge (S) x 1
Exact resource availability (Af) x 3
Uncertain resource availability (A) x 0
Exact resource price (Cf) x 3
Uncertain resource price (C) x 0
Exact process production (Pf) x 1
Uncertain process production (P) x 2
 For a total of 8 (5 fixed, and 3 uncertain)


In [176]:

# scales = TemporalScale(discretization_list=[1])

# Solar = Resource(name='Solar', cons_max=80, basis='MW', label='Solar Power')

# Wind = Resource(name='Wind', cons_max=85, basis='MW', label='Wind Power')

# Power = Resource(name='Power', basis='MW', label='Power generated', demand = True, varying = [VaryingResource.UNCERTAIN_DEMAND], varying_bounds  = (0.5,1))#(min(demand['COAST']), 1))

# H2O = Resource(name='H2O', cons_max=500, basis='tons', label='Water')

# O2 = Resource(name='O2', sell=True, basis='tons', label='Oxygen')

# H2 = Resource(name='H2', basis='tons', label='Hydrogen', demand= True) #, varying = [VaryingResource.UNCERTAIN_DEMAND], varying_bounds  = (0.6, 0.8))

# LiI = Process(name='LiI', storage=Power, vopex=1302, capex=0, fopex=0,  prod_max=20,
#             storage_loss=0.11,  label='Lithium-ion battery', basis='MW', store_max=30)

# WF = Process(name='WF', conversion={Wind: -1, Power: 0.85}, vopex=990, capex=0, fopex=0,
#             prod_max=95, label='Wind mill array', basis='MW', varying = [VaryingProcess.UNCERTAIN_CAPACITY]
#             , varying_bounds = (0.5,1))

# PV = Process(name='PV', conversion={Solar: -1, Power: 0.75}, vopex=567, capex=0, fopex=0,
#             prod_max=100, label='Solar PV', basis='MW', varying = [VaryingProcess.UNCERTAIN_CAPACITY]
#             , varying_bounds = (0.5,1))

# PEM = Process(name='PEM', conversion={Power: -1, H2: 0.3537, H2O: -3.1839}, capex=1.55*10**6, fopex=0,
#               vopex=0, prod_max=100, prod_min = 0, basis = 'tons', label = 'PEM' )

# region = Location(name='region', processes={LiI, WF, PV, PEM}, scales=scales, label='some region wo dispatchable')
# scenario = Scenario(name='example', demand={region: {Power:100, H2:20}}, network=region, scales=scales, label='example scenario')
# matrix = formulate(scenario=scenario, model_class=ModelClass.MPLP)
#     # print(matrix)
#     # return matrix



In [177]:
# matrix['c'] = numpy.array([[   10],
#        [   0.],
#        [   0.],
#        [   0.],
#        [1302.],
#        [   0.],
#        [ 567.],
#        [ 990.]])

In [178]:
solution= solve(matrix = matrix, interface = 'native', name = 'results', solver = 'ppopt') 

[]


In [179]:
len(solution.critical_regions)

2

In [180]:
solution.critical_regions[0]

Critical region with active set [0, 1, 2, 3, 4, 5, 6, 11]
The Omega Constraint indices are [0, 1, 2, 3, 4, 5]
The Lagrange multipliers Constraint indices are []
The Regular Constraint indices are [[1, 2, 4, 6, 8, 9, 11, 12], [8, 9, 12, 14, 16, 17, 19, 20]]
  x(θ) = Aθ + b 
 λ(θ) = Cθ + d 
  Eθ <= f
 A = [[-1.50399906e-14  1.73180428e-14  0.00000000e+00]
 [ 1.00000000e+02  3.00443277e-14  0.00000000e+00]
 [-1.89998858e-13  2.87378293e-13  0.00000000e+00]
 [ 2.21095382e-14  1.00000000e+02  0.00000000e+00]
 [ 1.17647059e+02 -8.82352941e+01  0.00000000e+00]
 [-5.36435356e-14  8.70852268e-14  0.00000000e+00]
 [-1.46704625e-14  1.00000000e+02  0.00000000e+00]
 [ 1.17647059e+02 -8.82352941e+01  0.00000000e+00]] 
 b = [[ 2.00000000e+01]
 [ 3.58611172e-15]
 [ 1.80033927e+02]
 [-3.63568309e-15]
 [ 6.65236408e+01]
 [ 5.65450947e+01]
 [ 2.70719625e-14]
 [ 6.65236408e+01]] 
 C = [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]] 
 d = [[-7.67516506e+03]

In [181]:
solution.evaluate(numpy.array([[0.3], [0.9], [0.9]]))

array([[ 20.        ],
       [ 30.        ],
       [180.03392706],
       [ 90.        ],
       [ 22.40599378],
       [ 56.54509471],
       [ 90.        ],
       [ 22.40599378]])

## Changes to OG problem to add MOO (emission objectives)

In [182]:
matrix['F']

array([[   0.,    0.,    0.],
       [   0.,    0.,    0.],
       [   0.,    0.,    0.],
       [   0.,    0.,    0.],
       [   0.,    0.,    0.],
       [   0.,    0.,    0.],
       [-100.,    0.,    0.],
       [   0.,    0.,    0.],
       [   0.,    0.,    0.],
       [   0.,    0.,    0.],
       [   0.,    0.,    0.],
       [   0.,  100.,    0.],
       [   0.,    0.,  100.],
       [   0.,    0.,    0.],
       [   0.,    0.,    0.],
       [   0.,    0.,    0.],
       [   0.,    0.,    0.],
       [   0.,    0.,    0.],
       [   0.,    0.,    0.],
       [   0.,    0.,    0.],
       [   0.,    0.,    0.]])

In [183]:
matrix['F'] = numpy.array([numpy.concatenate((i, numpy.array([0]*6)), axis = 0) for i in matrix['F']])

In [184]:
matrix['CRa'] = numpy.concatenate((numpy.eye(9),-numpy.eye(9)))

In [185]:
matrix['CRb'] = numpy.concatenate((numpy.array([[1] for i in range(9)]), numpy.array([[0] for i in range(9)])))

In [186]:
# add sum of thetas = 1
# matrix['A'] = numpy.vstack((matrix['A'], numpy.array([0]*8)))
# matrix['F'] = numpy.vstack((matrix['F'], numpy.concatenate((numpy.array([0]*3), numpy.array([-1]*6)), axis = 0)))
# matrix['b'] = numpy.vstack((matrix['b'], numpy.array([1])))

In [187]:
# material_cons_LiI = {LithiumCarbonateR: 20}
material_cons_WF = {Steel: 109.9, Cast_iron: 8.805, Concrete: 398.7}
material_cons_PV = {Glass: 70, Steel: 56, Concrete: 48, Aluminium: 19, Silicon: 7, Copper: 7}
material_cons_dict = {'PV': material_cons_PV, 'WF': material_cons_WF}#, 'LiI': material_cons_LiI}
material_cons_dict2 = {i: {m.name: n for m,n in j.items()} for i,j in material_cons_dict.items()}
objective_dict = {i: {m.name: {a: b*material_cons_dict2[i][m.name] for a,b in m.emission_potentials_dict.items()} for m,n in j.items()} for i, j in material_cons_dict.items()}

def calculate_total_potentials(data):
    totals = {}
    for process, materials in data.items():
        totals[process] = {}
        for material, impacts in materials.items():
            for impact, value in impacts.items():
                if impact not in totals[process]:
                    totals[process][impact] = 0
                totals[process][impact] += value
    return totals
objective_dict = calculate_total_potentials(objective_dict)


# Calculate and print total potentials
H1 = numpy.zeros((3,8))
# LiI_obj = numpy.array([[objective_dict['LiI'][i]] for i in ['gwp', 'odp', 'acid', 'eutt', 'eutf', 'eutm'] ])
# LiI_d_obj = numpy.array([[0] for i in ['gwp', 'odp', 'acid', 'eutt', 'eutf', 'eutm'] ])
PEM_obj = numpy.array([[0] for i in ['gwp', 'odp', 'acid', 'eutt', 'eutf', 'eutm'] ])
PV_obj = numpy.array([[objective_dict['PV'][i]] for i in ['gwp', 'odp', 'acid', 'eutt', 'eutf', 'eutm'] ])
WF_obj = numpy.array([[objective_dict['WF'][i]] for i in ['gwp', 'odp', 'acid', 'eutt', 'eutf', 'eutm'] ])

H2 = numpy.concatenate((numpy.zeros((6,5)), numpy.concatenate((PEM_obj, PV_obj, WF_obj), axis = 1)), axis = 1)

matrix['H'] = numpy.concatenate((H1, H2)).transpose()


In [188]:
# material_cons_LiI = {LithiumCarbonateR: 20}
# material_cons_WF = {Steel: 109.9, Cast_iron: 8.805, Concrete: 398.7}
# material_cons_PV = {Glass: 70, Steel: 56, Concrete: 48, Aluminium: 19, Silicon: 7, Copper: 7}
# material_cons_dict = {'PV': material_cons_PV, 'WF': material_cons_WF, 'LiI': material_cons_LiI}
# material_cons_dict2 = {i: {m.name: n for m,n in j.items()} for i,j in material_cons_dict.items()}
# objective_dict = {i: {m.name: {a: b*material_cons_dict2[i][m.name] for a,b in m.emission_potentials_dict.items()} for m,n in j.items()} for i, j in material_cons_dict.items()}
# def calculate_total_potentials(data):
#     totals = {}
#     for process, materials in data.items():
#         totals[process] = {}
#         for material, impacts in materials.items():
#             for impact, value in impacts.items():
#                 if impact not in totals[process]:
#                     totals[process][impact] = 0
#                 totals[process][impact] += value
#     return totals

# # Calculate and print total potentials
# objective_dict = calculate_total_potentials(objective_dict)
# H1 = numpy.zeros((3,8))
# LiI_obj = numpy.array([[objective_dict['LiI'][i]] for i in ['gwp', 'odp', 'acid', 'eutt', 'eutf', 'eutm'] ])
# LiI_d_obj = numpy.array([[0] for i in ['gwp', 'odp', 'acid', 'eutt', 'eutf', 'eutm'] ])
# PV_obj = numpy.array([[objective_dict['PV'][i]] for i in ['gwp', 'odp', 'acid', 'eutt', 'eutf', 'eutm'] ])
# WF_obj = numpy.array([[objective_dict['WF'][i]] for i in ['gwp', 'odp', 'acid', 'eutt', 'eutf', 'eutm'] ])
# H2 = numpy.concatenate((numpy.zeros((6,4)), numpy.concatenate((LiI_obj, LiI_d_obj, PV_obj, WF_obj), axis = 1)), axis = 1)
# matrix['H'] = numpy.concatenate((H1, H2)).transpose()

In [189]:
matrix['c'] = numpy.array([[0] for _ in range(8)])

In [190]:
solution_moo = solve(matrix = matrix, interface = 'native', name = 'results', solver = 'ppopt') 

[]


In [191]:
len(solution_moo.critical_regions)

3

\begin{align}
x(\theta) &= A\theta + b\\
\lambda(\theta) &= C\theta + d\\
\Theta &:= \{\forall \theta \in \mathbf{R}^m: E\theta \leq f\}
\end{align}

In [192]:
c

array([[   0.],
       [   0.],
       [   0.],
       [   0.],
       [   0.],
       [1550.],
       [ 567.],
       [ 990.]])

In [195]:
scenario.c_df

,0
Sf_H2,0.0
S_Power,0.0
Af_H2O,0.0
Af_Solar,0.0
Af_Wind,0.0
Pf_PEM,1550.0
P_PV,567.0
P_WF,990.0


In [197]:
x_1 = numpy.linspace(0.0001,0.9999, 50)
x_2 = 1 - x_1 

for index, _ in enumerate(x_1):
    x_sol = solution_moo.evaluate(numpy.array([[0.8], [0.9], [0.9], [x_1[index]], [0.001], [x_2[index]], [0.001], [0.001], [0.001]]))
    # print(x_sol)
    if x_sol is not None:
        obj_1 = matrix['H'].T[3]@x_sol 
        obj_2 = matrix['H'].T[5]@x_sol
        print(x_1[index], x_2[index], obj_1, obj_2)
        print(x_sol)
    

0.0001 0.9999 [83284113.55380619] [608118.42071576]
[[ 20.        ]
 [ 80.        ]
 [180.03392706]
 [ 80.06012628]
 [ 90.        ]
 [ 56.54509471]
 [ 80.06012628]
 [ 90.        ]]
0.020504081632653062 0.979495918367347 [83284113.55380619] [608118.42071576]
[[ 20.        ]
 [ 80.        ]
 [180.03392706]
 [ 80.06012628]
 [ 90.        ]
 [ 56.54509471]
 [ 80.06012628]
 [ 90.        ]]
0.04090816326530613 0.9590918367346939 [83284113.55380619] [608118.42071576]
[[ 20.        ]
 [ 80.        ]
 [180.03392706]
 [ 80.06012628]
 [ 90.        ]
 [ 56.54509471]
 [ 80.06012628]
 [ 90.        ]]
0.061312244897959195 0.9386877551020408 [83284113.55380619] [608118.42071576]
[[ 20.        ]
 [ 80.        ]
 [180.03392706]
 [ 80.06012628]
 [ 90.        ]
 [ 56.54509471]
 [ 80.06012628]
 [ 90.        ]]
0.08171632653061225 0.9182836734693878 [83284113.55380619] [608118.42071576]
[[ 20.        ]
 [ 80.        ]
 [180.03392706]
 [ 80.06012628]
 [ 90.        ]
 [ 56.54509471]
 [ 80.06012628]
 [ 90.     

In [194]:
solution_moo.program.display_latex()

$$\min_{x}\left[\begin{matrix}0\\0\\0\\0\\0\\0\\0\\0\end{matrix}\right]^T\left[\begin{matrix}x_{0}\\x_{1}\\x_{2}\\x_{3}\\x_{4}\\x_{5}\\x_{6}\\x_{7}\end{matrix}\right] + \left[\begin{matrix}\theta_{0}\\\theta_{1}\\\theta_{2}\\\theta_{3}\\\theta_{4}\\\theta_{5}\\\theta_{6}\\\theta_{7}\\\theta_{8}\end{matrix}\right]^{T}\left[\begin{matrix}0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 7.2e+05 & 0.01594 & 6.552e+03 & 1.015e+04 & 553.8 & 903.1\\0 & 0 & 0 & 2.849e+05 & 0.006309 & 928.5 & 2.991e+03 & 118.1 & 298.4\end{matrix}\right]\left[\begin{matrix}x_{0}\\x_{1}\\x_{2}\\x_{3}\\x_{4}\\x_{5}\\x_{6}\\x_{7}\end{matrix}\right]$$
$$\left[\begin{matrix}-1.0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & -1.0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 1.0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 1.0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 1.0 & 0 & 0 & 0\\0 & 0 & 0

$$\min_{x}\left[\begin{matrix}0\\0\\0\\0\\0\\1.55e+03\\567.0\\990.0\end{matrix}\right]^T\left[\begin{matrix}x_{0}\\x_{1}\\x_{2}\\x_{3}\\x_{4}\\x_{5}\\x_{6}\\x_{7}\end{matrix}\right] + \left[\begin{matrix}\theta_{0}\\\theta_{1}\\\theta_{2}\\\theta_{3}\\\theta_{4}\\\theta_{5}\\\theta_{6}\\\theta_{7}\\\theta_{8}\end{matrix}\right]^{T}\left[\begin{matrix}0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 7.2e+05 & 0.01594 & 6.552e+03 & 1.015e+04 & 553.8 & 903.1\\0 & 0 & 0 & 2.849e+05 & 0.006309 & 928.5 & 2.991e+03 & 118.1 & 298.4\end{matrix}\right]\left[\begin{matrix}x_{0}\\x_{1}\\x_{2}\\x_{3}\\x_{4}\\x_{5}\\x_{6}\\x_{7}\end{matrix}\right]$$
$$\left[\begin{matrix}-1.0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & -1.0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 1.0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 1.0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 1.0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 1.0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 1.0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 1.0\\-1.0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & -1.0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & -1.0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & -1.0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & -1.0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & -1.0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & -1.0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & -1.0\end{matrix}\right]\left[\begin{matrix}x_{0}\\x_{1}\\x_{2}\\x_{3}\\x_{4}\\x_{5}\\x_{6}\\x_{7}\end{matrix}\right]\leq\left[\begin{matrix}-20.0\\0\\500.0\\100.0\\100.0\\100.0\\0\\0\\0\\0\\0\\0\\0\\0\\0\\0\end{matrix}\right]+\left[\begin{matrix}0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\-100.0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 100.0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 100.0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\end{matrix}\right]\left[\begin{matrix}\theta_{0}\\\theta_{1}\\\theta_{2}\\\theta_{3}\\\theta_{4}\\\theta_{5}\\\theta_{6}\\\theta_{7}\\\theta_{8}\end{matrix}\right]$$
$$\left[\begin{matrix}-1.0 & 0 & 0 & 0 & 0 & 0.3537 & 0 & 0\\0 & -1.0 & 0 & 0 & 0 & -1.0 & 0.75 & 0.85\\0 & 0 & 1.0 & 0 & 0 & -3.184 & 0 & 0\\0 & 0 & 0 & 1.0 & 0 & 0 & -1.0 & 0\\0 & 0 & 0 & 0 & 1.0 & 0 & 0 & -1.0\end{matrix}\right]\left[\begin{matrix}x_{0}\\x_{1}\\x_{2}\\x_{3}\\x_{4}\\x_{5}\\x_{6}\\x_{7}\end{matrix}\right]=\left[\begin{matrix}0\\0\\0\\0\\0\end{matrix}\right]+\left[\begin{matrix}0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\end{matrix}\right]\left[\begin{matrix}\theta_{0}\\\theta_{1}\\\theta_{2}\\\theta_{3}\\\theta_{4}\\\theta_{5}\\\theta_{6}\\\theta_{7}\\\theta_{8}\end{matrix}\right]$$
$$\left[\begin{matrix}1.0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 1.0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 1.0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 1.0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 1.0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 1.0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 1.0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 1.0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1.0\\-1.0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & -1.0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & -1.0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & -1.0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & -1.0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & -1.0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & -1.0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & -1.0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & -1.0\end{matrix}\right]\left[\begin{matrix}\theta_{0}\\\theta_{1}\\\theta_{2}\\\theta_{3}\\\theta_{4}\\\theta_{5}\\\theta_{6}\\\theta_{7}\\\theta_{8}\end{matrix}\right]\leq\left[\begin{matrix}1.0\\1.0\\1.0\\1.0\\1.0\\1.0\\1.0\\1.0\\1.0\\0\\0\\0\\0\\0\\0\\0\\0\\0\end{matrix}\right]$$